In [ ]:
import pandas as pd
import glob
import json
import tweepy as tp
from datetime import datetime, date, timedelta

In [ ]:
access_token = 'FILL IN YOURS'
access_secret = 'FILL IN YOURS'
consumer_key = 'FILL IN YOURS'
consumer_secret = 'FILL IN YOURS'

auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth, wait_on_rate_limit=True)

In [ ]:
i = 310 #The day's number

filename = './CoronaTweets/corona_tweets_' + str(i) + '.csv'
    
#Printing when the collection starts
begin_date = date.today()
begin_time = datetime.now()
starting_date = begin_date.strftime("%d/%m/%Y")
starting_time = begin_time.strftime("%H:%M:%S")
print('Starting collecting file no. ' + str(i) + ' at ' + starting_date, starting_time)

In [ ]:
#Reading the file
corona_tweets = pd.read_csv(filename)
corona_tweets = corona_tweets.iloc[:, 0].drop_duplicates()
    
df = pd.DataFrame(columns = ['id', 'json']) #Empty main dataframe. Each batch dataframe will be appended here.

first = 0 #First tweet of the batch
last = 100 #Last tweet of the batch

while first < len(corona_tweets):
    
    list_id = [] #The future 'id' and 'json' columns
    list_json = []
    
    id_ = list(corona_tweets[first: last]) #The IDs of the tweets to collect 
    
    if first > (len(corona_tweets) - 100): #If the last batch has last less than 100 tweets
        last = len(corona_tweets)%first + first
        
    if first == last: #If there is only one tweet in the batch
        id_ = list(corona_tweets[first])
        
    statuses = api.statuses_lookup(id_, tweet_mode="extended")
    
    for status in statuses:
        list_id.append(status.id)
        list_json.append(json.dumps(status._json))
        
    temp_df = pd.concat([pd.Series(list_id,name='id'),pd.Series(list_json,name='json')], axis=1) #Batch dataframe
    
    df = df.append(temp_df) #Appending the batch dataframe to the main dataframe
        
    first += 100 #Updating the indices of first and last tweets of the batch
    last += 100
    
    if last%8000 == 0 or last%100 != 0: #Save files every 8000 tweets
        df.to_csv('./CollectedTweets/' + str(i) + '_corona_collected_tweets' + str(last) + '.csv')
        df = pd.DataFrame(columns = ['id', 'json'])
        
    
    #Printing when the collection ends
    end_date = date.today()
    end_time = datetime.now()
    finish_date = end_date.strftime("%d/%m/%Y")
    finish_time = end_now.strftime("%H:%M:%S")
    elapsed_time = end_time - begin_time
    print('Finishing collecting file no. ' + str(i) + ' at ' + finish_date, finish_time)
    print('Total time elapsed: ', str(elapsed_time.seconds//3600)+':'+str(elapsed_time.seconds%60))
    print()